# OpenAI API

In [1]:
with open("openai_api_key.txt", "rt") as f:
    key = f.read()
    print(f"Länge des Keys ist {len(key)}")
    print(key[:5])

Länge des Keys ist 51
sk-Da


In [5]:
from openai import OpenAI
client = OpenAI(api_key=key)

In [7]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
       {"role":"user",
        "content":"Hallo, ich heiße Jürgen. Sprichst du auch Deutsch?"}
    ]
  )

In [8]:
type(chat_completion)

openai.types.chat.chat_completion.ChatCompletion

In [9]:
chat_completion

ChatCompletion(id='chatcmpl-AW3DyLBPCMhA4MyQwlJPs9aOZm8Z3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hallo Jürgen! Ja, ich spreche auch Deutsch. Wie kann ich dir helfen?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732202254, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=18, prompt_tokens=22, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [18]:
chat_completion.choices[0].message.content

'Hallo Jürgen! Ja, ich spreche auch Deutsch. Wie kann ich dir helfen?'

## Textklassifikation

In [25]:
from openai import OpenAI
client = OpenAI(api_key=key)

ki_programmierung = """Du bist ein Anfrage-Klassifikator. Studierende werden dir Anfragen
schicken. Du sollst diese Anfragen in eine der folgenden 6 Kategorien stecken:
1. Studienberatungsanfrage
2. Adressänderung
3. Antrag auf Fristverlängerung für eine Prüfung
4. Abgabe Masterarbeit
5. Exmatrikulation
6. Keine der genannten Kategorien 1-5
Du sollst immer mit nur einer der Zahlen 1-6 antworten, je nachdem welche Anfrage vorliegt.
"""

anfrage = """Puh! Endlich geschafft. Habe über 100 Seiten geschrieben. Wo darf ich das Teil abgeben?"""

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
       {"role"    : "system",
        "content" : ki_programmierung,
       },
       {"role"    : "user",
        "content" : anfrage}
    ]
  )

antwort = chat_completion.choices[0].message.content

print(f"ANFRAGE: {anfrage}")
print(f"ANTWORT: {antwort}")

ANFRAGE: Puh! Endlich geschafft. Habe über 100 Seiten geschrieben. Wo darf ich das Teil abgeben?
ANTWORT: 4


In [23]:
antwort

'1'

## Stimmungsanalyse (Sentiment Analysis)

In [41]:
from openai import OpenAI
client = OpenAI(api_key=key)

ki_programmierung = """Du sollst die Stimmung in einem Text erkennen:
Dabei sollen die Zahlen 1-5 bedeuten:
Die Stimmung ist ...
1 - sehr negativ
2 - negativ
3 - neutral
4 - positiv
5 - sehr positiv
Antworte immer nur mit 1-5
"""

anfrage = """Die Vorlesung von Prof. Brauer ist doch lustig! Ich hatte nicht so viel erwartet.
So viele Themen und wir sind immer noch lange nicht durch!"""

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
       {"role"    : "system",
        "content" : ki_programmierung,
       },
       {"role"    : "user",
        "content" : anfrage}
    ]
  )

antwort = chat_completion.choices[0].message.content

print(f"ANFRAGE: {anfrage}")
print(f"ANTWORT: {antwort}")

ANFRAGE: Die Vorlesung von Prof. Brauer ist doch lustig! Ich hatte nicht so viel erwartet.
So viele Themen und wir sind immer noch lange nicht durch!
ANTWORT: 4


## Strukturierter Output

In [47]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(api_key=key)

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [48]:
type(event)

__main__.CalendarEvent

In [49]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [64]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(api_key=key)

ki_programmierung = """
Du sollst Anfragen von Studierenden analysieren.
Extrahiere für mich

- Name

- das Anliegen, z.B.
1. Studienberatungsanfrage
2. Adressänderung
3. Antrag auf Fristverlängerung für eine Prüfung
4. Abgabe Masterarbeit
5. Exmatrikulation
6. Keine der genannten Kategorien 1-5

- die Emotionalität: negativ, neutral, positiv

Die Emotionalität soll nur mit Smileys angegeben werden:
:-( bedeutet negativ
:-| neutral
:-) positiv
"""

anfrage1 = """
Mein Name ist Jürgen Brauer. Ich war bisher Dozent, will jetzt aber lieber an der Hochschule
Theoretische Quantenphysik studieren. Bin mit meinem Job hier nämlich sehr unzufrieden :-(
"""

anfrage2 = """Ich studiere Informatik. Bin von der Nebenstraße 1 in die Hauptstraße 2 umgezogen.
Können Sie das bitte endlich in ihrem System ändern? Heiße übrigens Homer Simpsons. Geht mir voll auf
die Nerven. Habe Ihnen diese Änderung schon vor 3 Jahren mitgeteilt und immer noch gehen
die Briefe an mich an die alte Adresse!!!"""

anfrage = anfrage2

class StudierendeAnfrage(BaseModel):
    name: str
    anliegen: str
    emotionalität: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": ki_programmierung},
        {"role": "user", "content": anfrage},
    ],
    response_format=StudierendeAnfrage,
)

event = completion.choices[0].message.parsed

In [65]:
event

StudierendeAnfrage(name='Homer Simpsons', anliegen='Adressänderung', emotionalität=':-(')

In [66]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(api_key=key)

ki_programmierung = """
Du sollst Beschreibungen von technischen Außendienstmitarbeitern analysieren.
Extrahiere für mich

- Problembeschreibung: Was war an dem Gerät kaputt?
- Lösungsbeschreibung: Wie wurde das Gerät repariert?
"""

beschreibung = """Bin zum Kunden Maier gefahren. Transformator X41 brummte im Technikraum.
Außerdem qualmte er. Deswegen lief die Hyperraumzeitmaschine nicht mehr einwandfrei.
Habe deswegen den X41 durch einen neuen X42 ausgetauscht. Zeitmaschine geht wieder.
Kunde ist ins Jahr 2050 gereist, um Rechnungsbezahlung zu vermeiden... Habe außerdem
mittags in der Kantine Spaghetti Bolognese gegessen. War echt lecker!
"""

class AussenDienstMitarbeiterBeschreibungsText(BaseModel):
    problem : str
    loesung : str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": ki_programmierung},
        {"role": "user", "content": beschreibung},
    ],
    response_format=AussenDienstMitarbeiterBeschreibungsText,
)

event = completion.choices[0].message.parsed

In [67]:
event

AussenDienstMitarbeiterBeschreibungsText(problem='Der Transformator X41 brummte und qualmte, wodurch die Hyperraumzeitmaschine nicht mehr einwandfrei funktionierte.', loesung='Der defekte Transformator X41 wurde durch einen neuen Transformator X42 ersetzt.')

In [1]:
exit

In [1]:
exit

In [1]:
type(exit)

IPython.core.autocall.ZMQExitAutocall